In [1]:
#import libraries
import pandas as pd
import numpy as np
import os

In [2]:
#create path
path = r'C:\Users\jacob\Documents\Data Analytics\Data Immersion\Achievement 6\Energy_GDP_Project'

In [3]:
#import gdp per capita data
gdp = pd.read_csv(os.path.join(path, '02 data', 'original data', 'gdp_per_capita.csv'))

In [4]:
gdp.head()

,country,year,combined key,gdp
0,Afghanistan,1990,"Afghanistan, 1990",NaN
1,Afghanistan,1991,"Afghanistan, 1991",NaN
2,Afghanistan,1992,"Afghanistan, 1992",NaN
3,Afghanistan,1993,"Afghanistan, 1993",NaN
4,Afghanistan,1994,"Afghanistan, 1994",NaN


In [5]:
gdp.shape

(8778, 4)

In [6]:
gdp.describe()

,year,gdp
count,8778.000000,8206.000000
mean,2006.000000,12103.702786
std,9.522447,20245.754235
min,1990.000000,22.850371
25%,1998.000000,1143.883563
50%,2006.000000,3777.668159
75%,2014.000000,14310.969895
max,2022.000000,234317.084800


In [7]:
#check for missing values
gdp.isnull().sum()

country           0
year              0
combined key      0
gdp             572
dtype: int64

In [8]:
#check for duplicats
gdp.duplicated().value_counts(dropna = False)

False    8778
Name: count, dtype: int64

In [9]:
#group data by country and then look for missing values by country
gdp_grouped = gdp.groupby(['country', 'year'])['gdp'].value_counts(dropna = False)

In [10]:
gdp_grouped

country      year  gdp        
Afghanistan  1990  NaN            1
             1991  NaN            1
             1992  NaN            1
             1993  NaN            1
             1994  NaN            1
                                 ..
Zimbabwe     2018  2269.177012    1
             2019  1421.868596    1
             2020  1372.696674    1
             2021  1773.920411    1
             2022  1266.996031    1
Name: count, Length: 8778, dtype: int64

In [11]:
gdp_grouped

country      year  gdp        
Afghanistan  1990  NaN            1
             1991  NaN            1
             1992  NaN            1
             1993  NaN            1
             1994  NaN            1
                                 ..
Zimbabwe     2018  2269.177012    1
             2019  1421.868596    1
             2020  1372.696674    1
             2021  1773.920411    1
             2022  1266.996031    1
Name: count, Length: 8778, dtype: int64

In [12]:
gdp_grouped_2 = gdp.groupby('year')['gdp'].value_counts(dropna = False)

In [13]:
gdp_grouped_2

year  gdp         
1990  NaN             40
      356.665379       2
      726.680909       2
      50.782948        1
      945.525160       1
                      ..
2022  11243.671230     1
      11091.305430     1
      10794.931860     1
      10616.061040     1
      10376.780250     1
Name: count, Length: 8173, dtype: int64

missing values begin to decrease as the years get closer and close. I am going to try creating moving average column to see if that will account for all of the missing values


that didn't super work because it back filled an average much higher than what it should have

i'm going to try a different method

In [14]:
#create a copy of gdp column
gdp['gdp_copy'] = gdp['gdp']

In [15]:
#check output
gdp.head()

,country,year,combined key,gdp,gdp_copy
0,Afghanistan,1990,"Afghanistan, 1990",NaN,NaN
1,Afghanistan,1991,"Afghanistan, 1991",NaN,NaN
2,Afghanistan,1992,"Afghanistan, 1992",NaN,NaN
3,Afghanistan,1993,"Afghanistan, 1993",NaN,NaN
4,Afghanistan,1994,"Afghanistan, 1994",NaN,NaN


In [16]:
#create a defintion for filling missing values with a moving average to impute future missing data points
def calculate_trend(group):
    group['trend'] = group['gdp'].rolling(window=4, min_periods=1).mean()
    group['gdp'] = group['gdp'].fillna(group['trend']).ffill()
    return group

In [17]:
#create new data frame with imputed values for future values
gdp_trend_filled = gdp.groupby('country', as_index= False).apply(calculate_trend)

In [18]:
gdp_trend_filled

country  year       combined key          gdp     gdp_copy  \
0   0     Afghanistan  1990  Afghanistan, 1990          NaN          NaN   
    1     Afghanistan  1991  Afghanistan, 1991          NaN          NaN   
    2     Afghanistan  1992  Afghanistan, 1992          NaN          NaN   
    3     Afghanistan  1993  Afghanistan, 1993          NaN          NaN   
    4     Afghanistan  1994  Afghanistan, 1994          NaN          NaN   
...               ...   ...                ...          ...          ...   
265 8773     Zimbabwe  2018     Zimbabwe, 2018  2269.177012  2269.177012   
    8774     Zimbabwe  2019     Zimbabwe, 2019  1421.868596  1421.868596   
    8775     Zimbabwe  2020     Zimbabwe, 2020  1372.696674  1372.696674   
    8776     Zimbabwe  2021     Zimbabwe, 2021  1773.920411  1773.920411   
    8777     Zimbabwe  2022     Zimbabwe, 2022  1266.996031  1266.996031   

                trend  
0   0             NaN  
    1             NaN  
    2             NaN  
    3             NaN  
    4             NaN  
...               ...  
265 8773  1573.350247  
    8774  1576.235103  
    8775  1563.962324  
    8776  1709.415673  
    8777  1458.870428  

[8778 rows x 6 columns]

In [19]:
#create a definition for missing past values
def calculate_trend_backward(group):
    group['trend'] = group['gdp'][::-1].rolling(window=3, min_periods=1).mean()[::-1]
    group['gdp'] = group['gdp'].bfill().fillna(group['trend'])
    return group

In [20]:
#create a new data from filling in missing past data points
gdp_trend_filled_2 = gdp_trend_filled.groupby('country', as_index=False).apply(calculate_trend_backward)

In [21]:
#check output
gdp_trend_filled_2

country  year       combined key          gdp     gdp_copy  \
0   0   0     Afghanistan  1990  Afghanistan, 1990   183.532775          NaN   
        1     Afghanistan  1991  Afghanistan, 1991   183.532775          NaN   
        2     Afghanistan  1992  Afghanistan, 1992   183.532775          NaN   
        3     Afghanistan  1993  Afghanistan, 1993   183.532775          NaN   
        4     Afghanistan  1994  Afghanistan, 1994   183.532775          NaN   
...                   ...   ...                ...          ...          ...   
265 265 8773     Zimbabwe  2018     Zimbabwe, 2018  2269.177012  2269.177012   
        8774     Zimbabwe  2019     Zimbabwe, 2019  1421.868596  1421.868596   
        8775     Zimbabwe  2020     Zimbabwe, 2020  1372.696674  1372.696674   
        8776     Zimbabwe  2021     Zimbabwe, 2021  1773.920411  1773.920411   
        8777     Zimbabwe  2022     Zimbabwe, 2022  1266.996031  1266.996031   

                    trend  
0   0   0             NaN  
        1             NaN  
        2             NaN  
        3             NaN  
        4             NaN  
...                   ...  
265 265 8773  1687.914094  
        8774  1522.828560  
        8775  1471.204372  
        8776  1520.458221  
        8777  1266.996031  

[8778 rows x 6 columns]

In [22]:
#check for missing values of gdp_trend_filled dataframe
gdp_trend_filled.isnull().sum()

country           0
year              0
combined key      0
gdp             474
gdp_copy        572
trend           520
dtype: int64

In [23]:
#chedk for missing values of gdp_trend_filled_2 dataframe
gdp_trend_filled_2.isnull().sum()

country           0
year              0
combined key      0
gdp             132
gdp_copy        572
trend           404
dtype: int64

In [24]:
#check shape
gdp_trend_filled_2.shape

(8778, 6)

In [25]:
#create a new dataframe removing the 4 countries (132 rows) that don't have any data
gdp_2 = gdp_trend_filled_2.dropna(subset = 'gdp')

In [26]:
#check shape
gdp_2.shape

(8646, 6)

In [27]:
#check math between shapes of the new and old dataframes
8778-8646

132

In [28]:
#drop gdp copy and trend columns
gdp_2.drop(columns = {'gdp_copy', 'trend'}, inplace = True)

C:\Users\jacob\AppData\Local\Temp\ipykernel_8484\3039755173.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdp_2.drop(columns = {'gdp_copy', 'trend'}, inplace = True)


In [29]:
gdp_2.head()

country  year       combined key         gdp
0 0 0  Afghanistan  1990  Afghanistan, 1990  183.532775
    1  Afghanistan  1991  Afghanistan, 1991  183.532775
    2  Afghanistan  1992  Afghanistan, 1992  183.532775
    3  Afghanistan  1993  Afghanistan, 1993  183.532775
    4  Afghanistan  1994  Afghanistan, 1994  183.532775

In [30]:
gdp_2.reset_index()

,level_0,level_1,level_2,country,year,combined key,gdp
0,0,0,0,Afghanistan,1990,"Afghanistan, 1990",183.532775
1,0,0,1,Afghanistan,1991,"Afghanistan, 1991",183.532775
2,0,0,2,Afghanistan,1992,"Afghanistan, 1992",183.532775
3,0,0,3,Afghanistan,1993,"Afghanistan, 1993",183.532775
4,0,0,4,Afghanistan,1994,"Afghanistan, 1994",183.532775
...,...,...,...,...,...,...,...
8641,265,265,8773,Zimbabwe,2018,"Zimbabwe, 2018",2269.177012
8642,265,265,8774,Zimbabwe,2019,"Zimbabwe, 2019",1421.868596
8643,265,265,8775,Zimbabwe,2020,"Zimbabwe, 2020",1372.696674
8644,265,265,8776,Zimbabwe,2021,"Zimbabwe, 2021",1773.920411


In [31]:
gdp_2.head()

country  year       combined key         gdp
0 0 0  Afghanistan  1990  Afghanistan, 1990  183.532775
    1  Afghanistan  1991  Afghanistan, 1991  183.532775
    2  Afghanistan  1992  Afghanistan, 1992  183.532775
    3  Afghanistan  1993  Afghanistan, 1993  183.532775
    4  Afghanistan  1994  Afghanistan, 1994  183.532775

In [32]:
gdp_2.shape

(8646, 4)

In [33]:
gdp_2

country  year       combined key          gdp
0   0   0     Afghanistan  1990  Afghanistan, 1990   183.532775
        1     Afghanistan  1991  Afghanistan, 1991   183.532775
        2     Afghanistan  1992  Afghanistan, 1992   183.532775
        3     Afghanistan  1993  Afghanistan, 1993   183.532775
        4     Afghanistan  1994  Afghanistan, 1994   183.532775
...                   ...   ...                ...          ...
265 265 8773     Zimbabwe  2018     Zimbabwe, 2018  2269.177012
        8774     Zimbabwe  2019     Zimbabwe, 2019  1421.868596
        8775     Zimbabwe  2020     Zimbabwe, 2020  1372.696674
        8776     Zimbabwe  2021     Zimbabwe, 2021  1773.920411
        8777     Zimbabwe  2022     Zimbabwe, 2022  1266.996031

[8646 rows x 4 columns]

In [34]:
gdp_3=gdp_2.reset_index()

In [35]:
gdp_3

,level_0,level_1,level_2,country,year,combined key,gdp
0,0,0,0,Afghanistan,1990,"Afghanistan, 1990",183.532775
1,0,0,1,Afghanistan,1991,"Afghanistan, 1991",183.532775
2,0,0,2,Afghanistan,1992,"Afghanistan, 1992",183.532775
3,0,0,3,Afghanistan,1993,"Afghanistan, 1993",183.532775
4,0,0,4,Afghanistan,1994,"Afghanistan, 1994",183.532775
...,...,...,...,...,...,...,...
8641,265,265,8773,Zimbabwe,2018,"Zimbabwe, 2018",2269.177012
8642,265,265,8774,Zimbabwe,2019,"Zimbabwe, 2019",1421.868596
8643,265,265,8775,Zimbabwe,2020,"Zimbabwe, 2020",1372.696674
8644,265,265,8776,Zimbabwe,2021,"Zimbabwe, 2021",1773.920411


In [36]:
#drop uneeded index columns
gdp_4 = gdp_3.drop(columns = ['level_0', 'level_1','level_2'])

In [37]:
gdp_4.head()

,country,year,combined key,gdp
0,Afghanistan,1990,"Afghanistan, 1990",183.532775
1,Afghanistan,1991,"Afghanistan, 1991",183.532775
2,Afghanistan,1992,"Afghanistan, 1992",183.532775
3,Afghanistan,1993,"Afghanistan, 1993",183.532775
4,Afghanistan,1994,"Afghanistan, 1994",183.532775


In [38]:
#check shape
gdp_4.shape

(8646, 4)

In [39]:
#check for missing data values
gdp_4.isnull().sum()

country         0
year            0
combined key    0
gdp             0
dtype: int64

In [40]:
#check descriptive stats
gdp_4.describe()

,year,gdp
count,8646.000000,8646.000000
mean,2006.000000,12366.147415
std,9.522455,20484.287913
min,1990.000000,22.850371
25%,1998.000000,1153.772085
50%,2006.000000,3852.266878
75%,2014.000000,15457.114572
max,2022.000000,234317.084800


In [ ]:
#export wragneld gdp_per_capita data
gdp_4.to_csv(os.path.join(path, '02 data', 'prepared data', 'gdp_per_capita_wrangled.csv'))